<div style="width:100%; overflow:hidden; background-color:#F1F1E6; padding: 10px; border-style: outset; color:#17469e">
    <div style="width: 80%; float: left;">
    <h2 align="center">Universidad de Sonora</h2>
    <hr style="border-width: 3px; border-color:#17469e">
          <h1>Reconocimiento de patrones: Preparación de los datos</h1>          
          <h4>Ramón Soto C. <a href="mailto:rsotoc@moviquest.com/">(rsotoc@moviquest.com)</a></h4>
    </div>
    <div style="float: right;">
    <img src="images/escudo_unison.png">
    </div>
</div>

## Caso de estudio: [*Stack Overflow 2018 Developer Survey*](https://www.kaggle.com/stackoverflow/stack-overflow-2018-developer-survey)

Como caso de estudio principal en el presente curso hemos seleccionado la encuesta de desarrolladores 2018 de *Stack Overflow* disponible en [Kaggle](https://www.kaggle.com). En este esta etapa realizaremos el análisis de agrupamientos.

### 4. Modelado - $k$-medias

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
Una vez inspeccionada la probable cantidad de clusters, en nuestro caso utilizando dendrogramas, ahora utilizamos la técnica de $k$-medias para identificar prototipos de clases. <br>Inicializamos el contexto y cargamos los datos:
</div>

In [1]:
"""
Reconocimiento de patrones: k-medias
"""

import pandas as pd
import numpy as np
import json
import pickle

import sys
from collections import Counter
from operator import itemgetter
from scipy.spatial.distance import squareform
from IPython.display import display, HTML

pd.set_option('display.max_columns', 130)
pd.set_option('max_colwidth', 80)

LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales

In [2]:
path = "Data sets/Stack Overflow Survey/"

# Recuperar encabezados de columnas en orden original
with open(path + 'survey_results_public_transformed.headers', 'rb') as file:  
    headers = pickle.load(file)

# Recuperar diccionarios... sólo por si se requieren
with open(path + 'survey_results_public_transformed.dicts', 'rb') as file:  
    dict_of_dicts = pickle.load(file)

with open(path + 'survey_results_public_transformed.json') as f:
    dict_json = json.load(f)
df = pd.DataFrame.from_dict(dict_json)

# Reordenar las columnas de acuerdo al orden original
df = df.reindex(headers, axis=1)

DATA_LEN = df.shape[0]

# Agregar una columna "cluster" inicializada a null 
df["Cluster"] = np.nan

In [3]:
var_str = ['Hobby', 'OpenSource', 'Country', 'Student', 'Employment', 'FormalEducation', 
         'UndergradMajor', 'CompanySize', 'YearsCoding', 'YearsCodingProf', 'UpdateCV', 
         'JobSatisfaction', 'CareerSatisfaction', 'HopeFiveYears', 'JobSearchStatus', 
         'LastNewJob', 'TimeFullyProductive', 'AgreeDisagree1', 'AgreeDisagree2', 
         'AgreeDisagree3', 'OperatingSystem', 'NumberMonitors', 'CheckInCode', 'AdBlocker', 
         'AdBlockerDisable', 'AdsAgreeDisagree1', 'AdsAgreeDisagree2', 'AdsAgreeDisagree3', 
         'AIDangerous', 'AIInteresting', 'AIResponsible', 'AIFuture', 'EthicsChoice', 
         'EthicsReport', 'EthicsResponsible', 'EthicalImplications', 'HoursComputer', 
         'StackOverflowRecommend', 'StackOverflowVisit', 'StackOverflowHasAccount', 
         'StackOverflowParticipate', 'StackOverflowJobs', 'StackOverflowDevStory', 
         'StackOverflowJobsRecommend', 'StackOverflowConsiderMember', 'HypotheticalTools1', 
         'HypotheticalTools2', 'HypotheticalTools3', 'HypotheticalTools4', 'WakeTime', 
         'HypotheticalTools5', 'HoursOutside', 'SkipMeals', 'Exercise', 'EducationParents', 
         'Age', 'Dependents', 'SurveyTooLong', 'SurveyEasy']
var_list = ['DevType', 'CommunicationTools', 'EducationTypes', 'SelfTaughtTypes', 
         'HackathonReasons', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 
         'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 
         'FrameworkWorkedWith', 'FrameworkDesireNextYear', 'IDE', 'Methodology', 
         'VersionControl', 'AdBlockerReasons', 'AdsActions', 'ErgonomicDevices', 
         'RaceEthnicity', 'LanguageWorkedWith']
var_ranks = ['AssessJob', 'AssessBenefits', 'JobContactPriorities', 'JobEmailPriorities', 
             'AdsPriorities']
var_float = 'ConvertedSalary'

def distance_qual(x, y):
    # Número de variables; si var_float es array, modificar "+ 1" por "+ len(var_float)"
    numvars = len(var_str) + len(var_list) + len(var_ranks) + 1
    
    distancia = abs(x.ConvertedSalary - y.ConvertedSalary)
    if pd.isnull(distancia):
        distancia = 0
        numvars -= 1
        
    for col in var_str:
        if x[col] != y[col]:
            distancia += 1
        
    for col in var_list:
        num_vars = len(x[col]) + len(y[col])
        d = 0
        if num_vars > 0:
            d = (2*len(set(x[col] + y[col])) - num_vars) / num_vars
        distancia += d

    for col in var_ranks:
        d = 0
        max_vars = max(len(x[col]), len(y[col]))
        if len(x[col]) != 0 and len(y[col]) != 0:
            for v in range(len(x[col])):
                if x[col][v] != y[col][v]:
                    d += 1
        else:
            d += max_vars
        
        if d != 0:
            d /= max_vars
        distancia += d
        
        #print(col, x[col], y[col], d)

    return distancia / numvars
    
def decode(dataframe):
    new_df = dataframe.copy(deep=True)
    
    for col in var_str:
        if col in list(dataframe) and col in dict_of_dicts:
            for index, row in dataframe.iterrows():
                value = dict_of_dicts[col][row[col]]
                new_df.at[clusters.index[index], col] = value
                
    for index, row in dataframe.iterrows():
        new_df.at[clusters.index[index], 'ConvertedSalary'] = row['ConvertedSalary'] * 200000
    
    for col in var_list:
        if col in list(dataframe):
            for index, row in dataframe.iterrows():
                values_list = row[col].copy()
                for i in range(len(values_list)):
                    values_list[i] = dict_of_dicts[col][values_list[i]]
                new_df.at[clusters.index[index], col] = values_list
                
    return new_df

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
A continuación ejecutamos el algoritmo de $k$-medias:
</div>

1) Definir el valor de $k$

In [4]:
NUM_CLUSTERS = 3

2) Seleccionar de manera arbitraria *k* puntos en el espacio de características como centros iniciales de los clusters (centroides o centros de masa).

In [5]:
# Inicializar los centroides
clusters = df.sample(n=NUM_CLUSTERS).reset_index(drop=True)

3) Asignar cada punto del conjunto de datos al cluster donde la distancia del punto al centroide es menor.

In [6]:
def update_clusters():
    changed = False
    cluster_col_index = df.shape[1] - 1
    
    # Asignar los puntos en la colección de datos al cluster del 
    # prototipo más cercano
    for index, row in df.iterrows():
        minDistance = LARGER_DISTANCE
        currentCluster = 0
        
        # Buscar la menor distancia del punto a un centroide
        for i, r in clusters.iterrows():
            dist = distance_qual(row, r)
            if(dist < minDistance):
                minDistance = dist
                currentCluster = i

        # Si hay cambio, realizarlo y levantar la bandera 'changed'
        if(pd.isnull(row['Cluster']) or row['Cluster'] != currentCluster):
            df.iloc[index, cluster_col_index] = currentCluster
            changed = True
            
    # Contabilizar los elementos en cada cluster   
    members = [0] * NUM_CLUSTERS
    for i in range(NUM_CLUSTERS):
        members[i] = df[df["Cluster"]==i].count()["Cluster"]
        if (TALK) : 
            print("El cluster ", i, " incluye ", members[i], "miembros.")
    if (TALK) : 
        print()
            
    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()

El cluster  0  incluye  27488 miembros.
El cluster  1  incluye  53870 miembros.
El cluster  2  incluye  17085 miembros.



4) Calcular los centroides a partir de los puntos en cada cluster. 

<div style="margin-top: 6px; margin-left: 50px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
El objetivo de este paso es obtener un vector representativo de la población en cada cluster. Sin embargo, el cálculo de centroides, en este caso, es complicado por tratarse de datos híbridos. 
<br><br>Los datos tienen un sólo dato numérico: 'ConvertedSalary'. Para esta variable, el valor representativo puede obetenerse como la media.
<br><br>El siguiente grupo de variables es el conjunto de variables nominales simples, etiquetadas como 'var_str'. En este caso, no tiene sentido una media, pero para cada una de estas variables, el valor más representativo lo proporciona la moda, que podemos obtener directamente del método 'DataFrame.mode()'.
<br><br>El grupo de variables 'var_list' son variables que describimos mediante listas de longitud variable; son el resultado de preguntas a las que el usuario podía responder con todas las opciones que aplicaran. Estas variables tienen, entonces, dos características: la cantidad de opciones utilizadas por el usuario y los valores específicos de ese listado. Entonces, para calcular la respuesta 'característica' hacemos dos calculos:
<ul>
<li>Obtener la longitud media de los vectores respuesta, y</li>
<li>Obtener los valores individuales más utilizados en los vectores respuestas</li>
</ul>
<br>El último grupo de variables a analizar es 'var_ranks'. Estas variables son listas ordenadas que son resultado de agrupar respuestas a preguntas múltiples bajo un mismo concepto y que describen un rankeo, como es el caso de las preguntas 'AssessJob1'-'AssessJob10'. En este caso, escogemos como respuesta representativa, un vector formado por los valores más comunes en cada posición. Lo ilustramos a continuación con la variable 'AssessJob':
</div>

In [7]:
# Seleccionar un conjunto de vectores de la población en el cluster '0'
df_clusterj = df[df["Cluster"] == 0].sample(15)

# Cantidad de elementos en el vector respuesta de cada columna en var_ranks.
# Por comodidad lo especificamos en esta lista
ranges = [11, 12, 6, 8, 8]

# ... obtener el índice a la lista de componentes en cada vector
# y el nombre de la columna
i = 0
col = 'AssessJob'

# Inicializar una matriz (lista de listas, en realidad), con tantos renglones como 
# componentes tiene el vector de la variable. Cada renglón tiene todos los valores 
# utilizados en cada posición del vector
vars = []
for j in range(ranges[i] - 1):
    vars.append([])

# Recorrer todos los elementos actualmente en el cluster para rellenar la matriz
# Imprimimos cada vector individual
print("Vectores individuales")
for index, row in df_clusterj.iterrows():
    # Si el vector de la variable no está vacío...
    print(index, row[col])
    if len(row[col]) > 0:
        # Para cada componente en el vector...
        for j in range(len(row[col])):
            # Si no es 0
            if row[col][j] != '0':
                # Agregarla al renglón actual en la matriz
                vars[j].append(row[col][j])

# Imprimir la matriz de ocurrencias
print("\nMatriz de ocurrencias:")
for r in vars:
    print(r)

# Contabilizar ocurrencias de cada componente. Crear una matriz con el orden para
# cada componente como renglones
print("\nMatriz de valores más comunes:")
most_commons = []
for j in range(ranges[i] - 1):
    counter = Counter(vars[j])
    most_commons.append(counter.most_common(ranges[i] - 1))
    print(most_commons[j])
print()

# Inicializar vector. Se escoge el valor más popular en la primera componente
vars_list = [most_commons[0][0][0]]
# Para cada componente a partir de la segunda...
for j in range(1, ranges[i] - 1):
    # Buscar la componente más común...
    for c in most_commons[j]:
        # Siempre y cuando no esté utilizada...
        if c[0] not in vars_list[:j]:
            # Agregarla al vector y...
            vars_list.append(c[0])
            # Dejar de buscar.
            break
print("Vector representativo\n", vars_list)

Vectores individuales
84961 ['9', '6', '7', '4', '1', '3', '2', '5', '10', '8']
48317 ['10', '7', '2', '4', '3', '1', '6', '5', '8', '9']
41539 ['9', '2', '7', '6', '1', '3', '8', '5', '10', '4']
77533 ['2', '10', '9', '1', '4', '3', '7', '6', '8', '5']
17943 ['6', '3', '5', '4', '8', '9', '1', '2', '7', '10']
60687 ['10', '2', '9', '8', '7', '4', '3', '1', '6', '5']
78853 ['8', '9', '1', '4', '7', '2', '5', '6', '3', '10']
16088 ['6', '8', '4', '3', '2', '10', '5', '9', '7', '1']
3509 ['10', '7', '3', '4', '6', '2', '8', '5', '9', '1']
26219 ['8', '5', '6', '3', '1', '4', '10', '2', '9', '7']
25104 ['10', '9', '2', '5', '4', '1', '3', '6', '7', '8']
50430 ['10', '8', '5', '1', '2', '6', '4', '3', '9', '7']
59021 ['7', '10', '8', '9', '2', '5', '4', '6', '1', '3']
20394 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
5830 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

Matriz de ocurrencias:
['9', '10', '9', '2', '6', '10', '8', '6', '10', '8', '10', '10', '7']
['6', '7', '2', '

<div style="margin-top: 6px; margin-left: 50px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
A continuación integramos estas ideas en una función:
</div>

In [8]:
def update_centroids():    
    for cl_j in range(NUM_CLUSTERS):
        means = [0] * (df.shape[1] - 1)
        
        # Seleccionar registros en el cluster cl_j
        df_clusterj = df[df["Cluster"] == cl_j]
        
        # Media en los datos numéricos
        col = 'ConvertedSalary'
        clusters.at[clusters.index[cl_j], col] = df_clusterj[col].mean()
        
        # Moda en las columnas 'simples' (en var_str)
        mode = df_clusterj[var_str].mode()
        for col in mode:
            clusters.at[clusters.index[cl_j], col] = mode[col].values[0]

        # Moda en las columnas con listas de longitud variable (en var_list)
        for col in var_list:
            mean_len = 0
            vars_list = []
            for index, row in df_clusterj.iterrows():
                mean_len += len(row[col])
                vars_list = vars_list + row[col]
            mean_len /= df_clusterj.shape[0]
            counter=Counter(vars_list)
            mean_list = []
            for v in counter.most_common(round(mean_len + 0.5)):
                mean_list.append(v[0])
            clusters.at[clusters.index[cl_j], col] = mean_list

            
        # Moda en las columnas con listas de longitud fija (en var_ranks)
        ranges = [11, 12, 6, 8, 8]
        # Para cada variable en var_list, obtener el número de componentes en el vector
        # y el nombre de la columna
        for i, col in zip(range(len(ranges)), var_ranks):
            # Inicializar una matriz (lista de listas, en realidad), con tantos renglones como 
            # componentes tiene el vector de la variable. Cada renglón tiene todos los valores 
            # utilizados en cada posición del vector
            vars = []
            for j in range(ranges[i] - 1):
                vars.append([])

            # Recorrer todos los elementos actualmente en el cluster para rellenar la matriz
            for index, row in df_clusterj.iterrows():
                # Si el vector de la variable no está vacío...
                if len(row[col]) > 0:
                    # Para cada componente en el vector...
                    for j in range(len(row[col])):
                        # Si no es 0
                        if row[col][j] != '0':
                            # Agregarla al renglón actual en la matriz
                            vars[j].append(row[col][j])

            # Contabilizar ocurrencias de cada componente. Crear una matriz con el orden para
            # cada componente como renglones
            most_commons = []
            for j in range(ranges[i] - 1):
                counter=Counter(vars[j])
                most_commons.append(counter.most_common(ranges[i] - 1))

            # Inicializar vector. Se escoge el valor más popular en la primera componente
            vars_list = [most_commons[0][0][0]]
            # Para cada componente a partir de la segunda...
            for j in range(1, ranges[i] - 1):
                # Buscar la componente más común...
                for c in most_commons[j]:
                    # Siempre y cuando no esté utilizada...
                    if c[0] not in vars_list[:j]:
                        # Agregarla al vector y...
                        vars_list.append(c[0])
                        # Dejar de buscar.
                        break
            clusters.at[clusters.index[cl_j], col] = vars_list
    return

# --------------------------
# Actualizar los centroides
update_centroids()

In [9]:
while(KEEP_WALKING):
    KEEP_WALKING = update_clusters()
    if (KEEP_WALKING):
        update_centroids()
    else :
        if (TALK) : 
            print ("No más cambios.")  

El cluster  0  incluye  33820 miembros.
El cluster  1  incluye  37874 miembros.
El cluster  2  incluye  26749 miembros.

El cluster  0  incluye  30881 miembros.
El cluster  1  incluye  42343 miembros.
El cluster  2  incluye  25219 miembros.

El cluster  0  incluye  33561 miembros.
El cluster  1  incluye  38992 miembros.
El cluster  2  incluye  25890 miembros.

El cluster  0  incluye  33956 miembros.
El cluster  1  incluye  38627 miembros.
El cluster  2  incluye  25860 miembros.

El cluster  0  incluye  33954 miembros.
El cluster  1  incluye  38629 miembros.
El cluster  2  incluye  25860 miembros.

El cluster  0  incluye  33953 miembros.
El cluster  1  incluye  38630 miembros.
El cluster  2  incluye  25860 miembros.

El cluster  0  incluye  33953 miembros.
El cluster  1  incluye  38630 miembros.
El cluster  2  incluye  25860 miembros.

No más cambios.


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
Los prototipos de las 3 clases obtenidos en este ejercicio son:
</div>

In [10]:
display(clusters)

,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,UpdateCV,ConvertedSalary,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,EducationParents,RaceEthnicity,Age,Dependents,SurveyTooLong,SurveyEasy,LanguageWorkedWith,AssessJob,AssessBenefits,JobContactPriorities,JobEmailPriorities,AdsPriorities,Cluster
0,1,1,USA,0,0,1,6,4,"[0, 12, 11, 15]",9,7,3,3,6,2,3,7,0.134423,"[8, 4, 5]",3,"[8, 1, 7]","[7, 5, 3]","[0, 4]",0,2,1,"[18, 27, 14, 5, 31]","[14, 17, 19]","[17, 13, 14]","[14, 22, 2]","[14, 2, 0]","[5, 1]","[5, 6]","[19, 17, 18]",3,2,"[0, 9, 4]","[1, 4]",2,2,3,"[6, 2]",1,1,0,"[3, 2]",1,3,3,1,1,0,2,2,10,2,2,4,1,2,5,2,3,2,3,3,4,6,2,0,3,[2],3,1,[6],1,0,0,2,"[18, 14, 5, 31, 1, 17]","[9, 8, 7, 2, 1, 3, 10, 4, 6, 5]","[1, 2, 3, 10, 9, 4, 7, 5, 11, 8, 6]","[2, 1, 5, 3, 4]","[1, 6, 7, 2, 3, 5, 4]","[1, 4, 2, 3, 6, 7, 5]",NaN
1,1,0,USA,0,0,1,6,8,"[0, 12, 11]",7,0,3,3,6,2,3,7,0.131948,"[8, 4]",3,"[8, 7, 5]","[5, 7, 0]",[0],0,1,1,"[18, 14, 27, 5]","[14, 19]","[14, 13]","[14, 22]","[14, 2, 0]","[5, 1]","[5, 6]","[18, 10, 19]",3,2,"[0, 9]","[1, 4]",2,2,3,"[6, 0]",1,1,0,"[3, 2]",0,3,3,1,1,0,2,2,10,5,2,4,2,2,5,2,3,2,3,4,3,5,2,2,3,[2],3,1,[6],1,0,0,2,"[18, 14, 5, 31, 17]","[9, 8, 7, 1, 2, 4, 10, 3, 6, 5]","[1, 2, 3, 10, 8, 4, 11, 6, 9, 7, 5]","[2, 1, 5, 3, 4]","[1, 5, 2, 3, 4, 7, 6]","[1, 4, 2, 5, 6, 7, 3]",NaN
2,1,1,IND,0,0,1,6,8,"[0, 12, 19]",7,11,5,5,2,1,5,7,0.016845,[8],0,"[8, 7]","[5, 7]",[4],0,0,4,"[18, 27, 14, 5]","[14, 19]","[14, 13]","[14, 2]","[14, 2]",[5],"[5, 6]","[15, 10]",3,1,[0],[1],2,2,3,[6],3,3,0,[3],1,0,3,1,1,0,2,2,10,5,2,4,2,2,5,2,3,1,1,1,1,5,2,0,3,[0],3,1,[6],0,0,1,2,"[14, 18, 5, 17]","[9, 8, 7, 1, 3, 4, 10, 2, 6, 5]","[1, 2, 3, 10, 6, 9, 11, 4, 8, 5, 7]","[2, 1, 5, 4, 3]","[1, 3, 7, 2, 5, 6, 4]","[1, 5, 3, 4, 6, 7, 2]",NaN


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
O bien, decodificando los valores:
</div>

In [11]:
dec_clusters = decode(clusters)
print(dec_clusters)

  Hobby OpenSource        Country Student          Employment  \
0   Yes        Yes  United States      No  Employed full-time   
1   Yes         No  United States      No  Employed full-time   
2   Yes        Yes          India      No  Employed full-time   

                            FormalEducation  \
0  Bachelor’s degree (BA, BS, B.Eng., etc.)   
1  Bachelor’s degree (BA, BS, B.Eng., etc.)   
2  Bachelor’s degree (BA, BS, B.Eng., etc.)   

                                                    UndergradMajor  \
0  Computer science, computer engineering, or software engineering   
1  Computer science, computer engineering, or software engineering   
2  Computer science, computer engineering, or software engineering   

          CompanySize  \
0  20 to 99 employees   
1                None   
2                None   

                                                                           DevType  \
0  [Back-end developer, Full-stack developer, Front-end developer, Mobile devel...

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
Estos resultados permiten realizar un análisis de conformación de los grupos, así como revizar la utilidad de las variables registradas. Obsérvese que diversas varaibles no cambian de valor para las 3 prototipos, lo cual, a este nivel, indicaría que no aportan poder discriminante. <br><br>Una exploración más detallada permitiría descartar variables o, después de una revisión de los objetivos del negocio, un replanteamiento del procedimiento para recabar los datos.
<br><br>Una manera de evaluar la calidad de los clusters obtenidos es evaluar la separación entre los prototipos. Distancias muy pequeñas entre dos prototipos indicarían la conveniencia de unificar los clusters correspondientes:
</div>

In [12]:
list_array = []
for index, row in clusters.iterrows():
    for i in range(index + 1, clusters.shape[0]):
        list_array.append(distance_qual(clusters.iloc[index], clusters.iloc[i]))
print(squareform(list_array))

[[ 0.          0.20316469  0.40188578]
 [ 0.20316469  0.          0.37451263]
 [ 0.40188578  0.37451263  0.        ]]


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
Repetimos el análisis, utilizando ahora $k=5$:
</div>

In [13]:
NUM_CLUSTERS = 5

clusters = df.sample(n=NUM_CLUSTERS).reset_index(drop=True)

KEEP_WALKING = True
while(KEEP_WALKING):
    KEEP_WALKING = update_clusters()
    if (KEEP_WALKING):
        update_centroids()
    else :
        if (TALK) : 
            print ("No más cambios.")
            
display(clusters)

El cluster  0  incluye  13288 miembros.
El cluster  1  incluye  18226 miembros.
El cluster  2  incluye  9063 miembros.
El cluster  3  incluye  34485 miembros.
El cluster  4  incluye  23381 miembros.

El cluster  0  incluye  19115 miembros.
El cluster  1  incluye  13247 miembros.
El cluster  2  incluye  20147 miembros.
El cluster  3  incluye  26849 miembros.
El cluster  4  incluye  19085 miembros.

El cluster  0  incluye  21735 miembros.
El cluster  1  incluye  15754 miembros.
El cluster  2  incluye  19111 miembros.
El cluster  3  incluye  25742 miembros.
El cluster  4  incluye  16101 miembros.

El cluster  0  incluye  21078 miembros.
El cluster  1  incluye  16128 miembros.
El cluster  2  incluye  18781 miembros.
El cluster  3  incluye  26219 miembros.
El cluster  4  incluye  16237 miembros.

El cluster  0  incluye  21069 miembros.
El cluster  1  incluye  16121 miembros.
El cluster  2  incluye  18575 miembros.
El cluster  3  incluye  26141 miembros.
El cluster  4  incluye  16537 miembro

,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,UpdateCV,ConvertedSalary,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,EducationParents,RaceEthnicity,Age,Dependents,SurveyTooLong,SurveyEasy,LanguageWorkedWith,AssessJob,AssessBenefits,JobContactPriorities,JobEmailPriorities,AdsPriorities,Cluster
0,1,1,IND,0,0,1,6,4,"[0, 12, 11, 15]",7,7,3,3,6,2,3,7,0.103656,"[8, 4, 7]",0,"[8, 1, 7]","[7, 5, 0]","[0, 4]",0,0,2,"[18, 27, 14, 5, 31]","[14, 19, 13]","[13, 14, 17]","[14, 2, 0]","[14, 2, 0]","[5, 1]","[5, 6]","[15, 19, 10]",3,2,"[0, 9]","[1, 4]",2,2,3,"[6, 0]",1,1,0,"[3, 0]",1,3,3,1,1,0,2,2,10,5,2,0,2,0,5,2,3,3,4,3,4,6,2,0,3,[0],3,1,[6],1,0,1,2,"[18, 14, 5, 31, 17, 1]","[9, 8, 7, 1, 2, 4, 10, 3, 6, 5]","[1, 2, 3, 10, 8, 9, 11, 4, 7, 6, 5]","[2, 1, 5, 3, 4]","[1, 6, 7, 2, 3, 5, 4]","[1, 4, 2, 5, 6, 7, 3]",0.0
1,1,1,USA,0,0,1,6,8,"[0, 12, 11]",7,11,5,5,0,1,3,7,0.066358,"[8, 4]",0,"[8, 1]","[7, 5]",[0],0,2,4,"[18, 27, 14, 5]","[14, 17]","[17, 14]","[14, 22]","[14, 2]",[5],"[5, 6]","[17, 19, 18]",3,2,"[0, 9]",[1],2,2,3,[6],1,1,3,[3],0,3,3,1,1,0,2,2,10,5,2,4,1,2,5,2,2,2,2,2,2,6,2,2,3,[0],3,1,[6],1,0,1,2,"[18, 14, 5, 31, 1, 27]","[9, 8, 7, 1, 2, 3, 10, 4, 6, 5]","[1, 2, 3, 10, 9, 4, 7, 5, 11, 8, 6]","[5, 1, 4, 3, 2]","[1, 4, 2, 3, 5, 7, 6]","[1, 4, 2, 3, 6, 7, 5]",2.0
2,1,0,USA,0,0,1,6,8,"[0, 12, 11]",7,0,3,3,6,2,3,7,0.106492,"[5, 8]",3,"[8, 7]","[5, 7, 3]",[0],0,0,1,"[18, 3, 31, 27]","[19, 14]","[19, 14]","[22, 14]","[22, 14]",[0],"[0, 5]","[18, 10, 19]",3,2,"[0, 9]","[1, 5]",2,2,3,"[6, 2]",1,0,0,[3],2,3,3,1,0,0,2,2,10,5,2,4,1,1,5,2,3,3,3,3,3,5,2,2,3,[2],3,2,[6],0,0,0,2,"[14, 18, 5, 31, 3]","[8, 9, 6, 1, 2, 3, 10, 4, 7, 5]","[1, 2, 3, 10, 9, 4, 7, 5, 11, 8, 6]","[2, 1, 5, 3, 4]","[1, 4, 3, 2, 5, 7, 6]","[1, 5, 2, 3, 6, 7, 4]",0.0
3,1,0,USA,0,0,1,6,4,"[0, 12, 11]",9,7,3,3,6,2,3,7,0.172920,"[8, 4, 5]",3,"[8, 7, 5]","[7, 5, 0]",[0],0,1,1,"[18, 14, 27, 5, 31]","[14, 19, 17]","[17, 13, 14]","[14, 22, 0]","[14, 0, 2]","[5, 1]","[5, 6]","[19, 18, 10]",3,2,"[0, 9, 4]","[1, 4]",2,2,3,"[6, 2]",1,1,0,"[3, 2]",0,3,3,1,1,3,2,2,10,2,2,4,2,2,5,2,3,2,3,4,4,6,2,2,3,[2],3,1,[6],1,0,0,4,"[18, 14, 5, 31, 1, 17]","[9, 8, 7, 2, 1, 4, 10, 3, 6, 5]","[1, 2, 3, 10, 9, 4, 11, 5, 8, 7, 6]","[2, 1, 5, 3, 4]","[1, 5, 2, 3, 4, 7, 6]","[1, 4, 2, 5, 6, 7, 3]",1.0
4,1,0,IND,0,0,1,6,8,"[0, 19, 11]",7,11,5,5,2,1,5,7,0.014726,[8],3,[8],[5],[4],0,2,2,"[18, 27, 14]",[14],"[14, 13]","[14, 2]","[2, 14]",[5],[5],"[10, 15]",3,1,[0],[1],2,1,0,[0],1,1,0,[2],1,3,3,1,0,0,1,2,10,2,2,4,1,2,5,2,3,4,4,4,4,5,1,0,3,[0],3,1,[6],0,0,0,2,"[14, 5, 18, 17]","[1, 9, 6, 2, 3, 4, 10, 5, 8, 7]","[1, 11, 2, 10, 9, 5, 8, 3, 7, 6, 4]","[2, 1, 5, 4, 3]","[1, 3, 7, 2, 6, 5, 4]","[1, 5, 3, 4, 6, 7, 2]",2.0


In [14]:
dec_clusters = decode(clusters)
print(dec_clusters)

  Hobby OpenSource        Country Student          Employment  \
0   Yes        Yes          India      No  Employed full-time   
1   Yes        Yes  United States      No  Employed full-time   
2   Yes         No  United States      No  Employed full-time   
3   Yes         No  United States      No  Employed full-time   
4   Yes         No          India      No  Employed full-time   

                            FormalEducation  \
0  Bachelor’s degree (BA, BS, B.Eng., etc.)   
1  Bachelor’s degree (BA, BS, B.Eng., etc.)   
2  Bachelor’s degree (BA, BS, B.Eng., etc.)   
3  Bachelor’s degree (BA, BS, B.Eng., etc.)   
4  Bachelor’s degree (BA, BS, B.Eng., etc.)   

                                                    UndergradMajor  \
0  Computer science, computer engineering, or software engineering   
1  Computer science, computer engineering, or software engineering   
2  Computer science, computer engineering, or software engineering   
3  Computer science, computer engineering, or 

In [15]:
from scipy.spatial.distance import squareform

list_array = []
for index, row in clusters.iterrows():
    for i in range(index + 1, clusters.shape[0]):
        list_array.append(distance_qual(clusters.iloc[index], clusters.iloc[i]))
print(squareform(list_array))

[[ 0.          0.32435868  0.35016832  0.27076139  0.40542363]
 [ 0.32435868  0.          0.37157988  0.32817838  0.42098431]
 [ 0.35016832  0.37157988  0.          0.30131627  0.44420158]
 [ 0.27076139  0.32817838  0.30131627  0.          0.45447888]
 [ 0.40542363  0.42098431  0.44420158  0.45447888  0.        ]]


<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
En esos resultados, se repite, en gran medida, el comportamiento ya observado con 3 clusters.
</div>